# Nordrhein-Westfalen

Every federal state is represented by its own input directory and is processed into a NUTS level 2 directory containing a sub-folder for each discharge location. These folder names are derived from NUTS and reflect the CAMELS id. The NUTS level 2 code for Nordrhein-Westfalen is `DEA`.

To pre-process the data, you need to write (at least) two functions. One should extract all metadata and condense it into a single `pandas.DataFrame`. This is used to build the folder structure and derive the ids.
The second function has to take an id, as provided by the state authorities, called `provider_id` and return a `pandas.DataFrame` with the transformed data. The dataframe needs the three columns `['date', 'q' | 'w', 'flag']`.

For easier and unified output handling, the `camelsp` package contains a context object called `Bundesland`. It takes a number of names and abbreviations to identify the correct federal state and returns an object that holds helper and save functions.

The context saves files as needed and can easily be changed to save files with different strategies, ie. fill missing data with NaN, merge data into a single file, create files for each variable or pack everything together into a netcdf.

In [1]:
import pandas as pd
import numpy as np
from pandas.errors import ParserError
import os
from pprint import pprint
from tqdm import tqdm
from typing import Union, Dict
import patoolib
from glob import glob
from datetime import datetime as dt
from dateparser import parse
import warnings
from io import StringIO

from camelsp import Bundesland


The context can also be instantiated as any regular Python class, ie. to load only the default input data path, that we will user later.

In [2]:
# the context also makes the input path available, if camelsp was install locally
BASE = Bundesland('NRW').input_path
BASE

'/home/alexander/Github/camels/camelsp/input_data/NRW_Nordrhein-Westfalen'

In [17]:
# remove everything from BASE folder except Q&W.rar to ensure that data is extracted freshly and is up to date
for f in glob(f"{BASE}/*[!*.rar]"):
    os.remove(f)

# extract rar archive
patoolib.extract_archive(f"{BASE}/Q&W.rar", outdir=BASE)

patool: Extracting /home/alexander/Github/camels/camelsp/input_data/NRW_Nordrhein-Westfalen/Q&W.rar ...
patool: running /usr/bin/unrar x -- /home/alexander/Github/camels/camelsp/input_data/NRW_Nordrhein-Westfalen/Q&W.rar
patool:     with cwd='/home/alexander/Github/camels/camelsp/input_data/NRW_Nordrhein-Westfalen'
patool: ... /home/alexander/Github/camels/camelsp/input_data/NRW_Nordrhein-Westfalen/Q&W.rar extracted to `/home/alexander/Github/camels/camelsp/input_data/NRW_Nordrhein-Westfalen'.


'/home/alexander/Github/camels/camelsp/input_data/NRW_Nordrhein-Westfalen'

In [21]:
with Bundesland('NRW') as bl:
    metadata = pd.read_excel(os.path.join(bl.input_path, 'Stammdaten_CAMELS.xlsx'))

metadata

,NAME,ORT,NULLPUNKT,STATIONIER,GEBFLAECHE,UTMZONE,KOORDX,KOORDY,KOMMENTAR,GEBIETSKEN,Gewässerkennzahl,Gewässer
0,Ahmsen,4639000000100,64.285,27.158,593.00,32U,479549.677600,5.771202e+06,Grundmessstelle des Landes (GL),4639.0,46,Werre
1,Albersloh,3259000000100,48.678,27.470,321.58,32U,412463.350631,5.748891e+06,NaN,3259.0,32,Werse
2,Altena,2766930000100,154.225,29.700,1190.00,32U,407683.711900,5.682847e+06,Talsperrenbeeinflussung ab 1968 (Biggetalsperre),276693.0,2766,Lenne
3,Altenbeken 2,2781610000200,215.958,11.990,20.50,32U,494359.426900,5.734473e+06,Grundmeßstelle des Landes (GL),278161.0,27816,Beke
4,Altenburg 1,2823900000200,82.651,62.440,958.76,32U,315309.586100,5.641695e+06,Grundmessstelle,28239.0,282,Rur
...,...,...,...,...,...,...,...,...,...,...,...,...
214,Westtuennen,2786700000100,57.572,3.970,414.90,32U,421634.000000,5.724518e+06,Grundmessstelle des Landes (GL),NaN,2786,Ahse
215,Wetter_Wengern_1,2769169000100,84.805,0.480,17.58,32U,384656.787100,5.695710e+06,Grundmeßstelle des Landes (GL),2769169.0,276916,Elbsche
216,Wettringen B70,9286291000300,41.504,6.330,175.07,32U,385480.262390,5.786270e+06,NaN,928629.0,92862,Steinfurter Aa
217,Wt-Kluserbrücke,2736510000100,142.226,49.240,337.82,32U,371494.000000,5.679856e+06,Durch mehrere Talsperren beeinflusst. Seit 01....,273651.0,2736,Wupper


### Metadata reader

Define the function that extracts / reads and eventually merges all metadata for this federal state. You can develop the function here, without using the Bundesland context and then later use the context to pass extracted metadata. The Context has a function for saving *raw* metadata, that takes a `pandas.DataFrame` and needs you to identify the id column.
Here, *raw* refers to provider metadata, that has not yet been transformed into the CAMELS-de Metadata schema.

In [22]:
# the id column will be ORT
id_column = 'ORT'

## file extract and parse

Here, we need to process the filename as the `'Ort'` is contained in the filename. Looks like the metadata header is **always** to line 32, indicating a finished header by `YTYP;`. Verify this.

In [25]:
for fname in glob(os.path.join(BASE, 'Datenanfrage_CAMELS_*')):
    df = pd.read_csv(fname, encoding='latin1', sep=';', usecols=[0,1], nrows=32, header=None)
    if df.iloc[31, 0] != 'YTYP':
        print(fname)
        

That's will make our lifes way easier. Now go for all:

In [26]:
# get all file names
filelist = glob(os.path.join(BASE, 'Datenanfrage_CAMELS_*'))

# container for meta-header and dataframes
meta = []
data = []

# go for each file
for fname in tqdm(filelist):
    # open
    with open(fname, 'rb') as f:
        txt = f.read().decode('latin1')
    
    # split header
    header = txt.splitlines()[:32]
    
    # build the meta by hand
    tups = [l.split(';') for l in header[:-1]]
    meta_dict = {t[0]: t[1] for t in tups}

    # check the parameter
    if meta_dict['Parameter'] == 'Wasserstand':
        variable = 'w'
    elif meta_dict['Parameter'] == 'Abfluss':
        variable = 'q'
    else:
        raise RuntimeError(f"Unknown Parameter: {meta_dict['Parameter']}")

    meta.append(meta_dict)
    
    # now get the body
    body = txt.splitlines()[32:]

    # now this stupid check
    second_header = [i for i, l in enumerate(body) if l.startswith('Station')]
    if len(second_header) > 0:
        # THERE IS A SECOND HEADER IN THE FILE !!!! come on!
        body = body[:second_header[0]]
    
    # write to buffer
    buffer = StringIO('\n'.join(body))
    buffer.seek(0)
    
    # read from memory
    df_data = pd.read_csv(buffer, sep=';', usecols=[0,1], skiprows=32, decimal=',', header=None, na_values='LUECKE', parse_dates=[0])
    
    df_data.columns = ['date', variable]
    df_data['flag'] = np.NaN
    
    # append
    data.append(df_data)
    
    
print(f"Parsed {len(meta)} metadata headers and {len(data)} data files")

100%|██████████| 437/437 [08:29<00:00,  1.17s/it]

Parsed 437 metadata headers and 437 data files


That was really stupid. Ok. Check the metadata from the data files:

In [32]:
extra = pd.DataFrame(meta).drop('Gewässer', axis=1)
extra.head()

,Station,Stationsnummer,Unterbezeichnung,Einzugsgebiet,Pegelnullpunkt,Parameter,Einheit,Aussage,Lebenslauf,Zeitangabe,...,PARMERKMAL,PUBLIZIERT,QUELLE,REIHENART,VERSION,X,XDISTANZ,XEINHEIT,XFAKTOR,Y
0,Kreuztal,2721459000100,,"63,40 km²","273,894 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('2721459000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,,True,P,Z,0,429430,T,,1,5645719
1,Liesborn,2784650000100,,"66,30 km²","73,405 mNHN (aktuell)",Wasserstand,cm,Mittelwert,"MITTEL('2784650000100.wk4',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,,True,P,Z,0,449139,T,,1,5729507
2,Veert,2850000000200,,"0,00 km²","22,376 mNHN (aktuell)",Wasserstand,cm,Mittelwert,"MITTEL('2850000000200.wk2',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,,True,P,Z,0,313366,T,,1,5710953
3,Raumland,4281490000100,,"84,70 km²","400,254 mNHN (aktuell)",Wasserstand,cm,Mittelwert,"MITTEL('4281490000100.wk3',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,,True,P,Z,0,456965,T,,1,5653501
4,Hörstel,3448390000200,,"88,66 km²","40,018 mDHHN92 (aktuell)",Wasserstand,cm,Mittelwert,"MITTEL('3448390000200.wk3',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,,True,P,Z,0,403784,T,,1,5797597


Now we have to left-join the data, as each Stationsnummer exists twice. Thus, it is only the combination of Stationsnummer and variable, that makes the data unique

In [34]:
metadata = extra.drop('KOMMENTAR', axis=1).join(metadata.set_index(metadata.ORT.astype(str)), on='Stationsnummer', how='left')
metadata.head()

ValueError: columns overlap but no suffix specified: Index(['Station', 'Stationsnummer', 'Unterbezeichnung', 'Einzugsgebiet',
       'Pegelnullpunkt', 'Parameter', 'Einheit', 'Aussage', 'Lebenslauf',
       'Zeitangabe', 'DEFART', 'FTOLERANZ', 'GUELTBIS', 'GUELTVON',
       'HAUPTREIHE', 'HERKUNFT', 'HOEHE', 'MESSGENAU', 'NWGRENZE',
       'PARMERKMAL', 'PUBLIZIERT', 'QUELLE', 'REIHENART', 'VERSION', 'X',
       'XDISTANZ', 'XEINHEIT', 'XFAKTOR', 'Y'],
      dtype='object')

In [35]:
# build and id column
metadata['ID'] = metadata.apply(lambda r: r.Stationsnummer + '_' + r.Parameter, axis=1)

id_column = 'ID'

### Finally run

Now, the Q and W data can be extracted. The cool thing is, that all the id creation, data creation, merging and the mapping from our ids to the original ids and files is done by the context. This is helpful, as we less likely screw something up.

In [36]:
with Bundesland('NRW') as bl:
    # save the metadata
    bl.save_raw_metadata(metadata, 'Stationsnummer', overwrite=True)

    # for reference, call the nuts-mapping as table
    nuts_map = bl.nuts_table
    print(nuts_map.head())

    
    with warnings.catch_warnings(record=True) as warns:
        for m, df in tqdm(zip(meta, data), total=len(meta)):
            # check the meta
            provider_id = str(m['Stationsnummer'])
            bl.save_timeseries(df, provider_id)

        # check if there were warnings (there are warnings)
        if len(warns) > 0:
            log_path = bl.save_warnings(warns)
            print(f"There were warnings during the processing. The log can be found at: {log_path}")


    nuts_id    provider_id                              path
0  DEA10000  2721459000100  ./DEA/DEA10000/DEA10000_data.csv
1  DEA10010  2784650000100  ./DEA/DEA10010/DEA10010_data.csv
2  DEA10020  2850000000200  ./DEA/DEA10020/DEA10020_data.csv
3  DEA10030  4281490000100  ./DEA/DEA10030/DEA10030_data.csv
4  DEA10040  3448390000200  ./DEA/DEA10040/DEA10040_data.csv


100%|██████████| 437/437 [00:21<00:00, 20.64it/s]
